<a href="https://colab.research.google.com/github/katyk20/distance-education-highered/blob/main/Notebooks/Create_Model_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mount G-Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import timeit
import re

In [ ]:
# ! git clone https://github.com/katyk20/distance-education-highered.git

Cloning into 'DistanceLearning_Capstone'...
remote: Enumerating objects: 792, done.
remote: Counting objects: 100% (434/434), done.
remote: Compressing objects: 100% (240/240), done.
remote: Total 792 (delta 288), reused 297 (delta 194), pack-reused 358
Receiving objects: 100% (792/792), 79.08 MiB | 14.46 MiB/s, done.
Resolving deltas: 100% (404/404), done.


In [ ]:
# %cd /content/istance-education-highered.git
# !git pull

/content/DistanceLearning_Capstone
Already up to date.


IPEDS Crosswalk with SOC Codes

In [ ]:
#read in file that crosswalks IPEDS CIP codes with BLS SOC codes
#Extract code columns using sheet SOC-CIP
crosswalk=pd.read_excel('/content/drive/MyDrive/Capstone_Winter2024/TABULAR DATA/CIP2020_SOC2018_Crosswalk.xlsx', sheet_name="SOC-CIP")


In [ ]:
# # crosswalk.head()
cols_keep =['SOC2018Code','CIP2020Code']
codes = crosswalk[cols_keep]
codes= codes.rename(columns ={'CIP2020Code':'CIPCODE'})


In [ ]:
codes.head()
# # len(codes)

,SOC2018Code,CIPCODE
0,11-1011,44.0401
1,11-1011,52.0101
2,11-1011,52.0201
3,11-1011,52.0206
4,11-1011,52.0701


Merging SOC Codes based on CIPCodes in depprogramsoffered (file of distance educaton degrees conferred based on CIP field of study)

In [ ]:
#read in deprogram completes file 2018-2022
de_degrees1822 = pd.read_csv('/content/drive/MyDrive/Capstone_Winter2024/PROJECT DATASETS/depprogramsoffered18_22.csv')


In [ ]:
# de_degrees1822.isnull().sum()

In [ ]:

de_degrees1822.columns

Index(['UNITID', 'INSTNM', 'YEAR', 'IALIAS', 'STABBR', 'OBEREG', 'ICLEVEL',
       'CONTROL', 'DEGGRANT', 'LONGITUD', 'LATITUDE', 'CIPCODE', 'PTOTAL',
       'PTOTALDE', 'PTOTALDES', 'PASSOC', 'PASSOCDE', 'PASSOCDES', 'PBACHL',
       'PBACHLDE', 'PBACHLDES', 'PMASTR', 'PMASTRDE', 'PMASTRDES', 'PDOCRS',
       'PDOCRSDE', 'PDOCRSDES', 'PDOCPP', 'PDOCPPDE', 'PDOCPPDES', 'PDOCOT',
       'PDOCOTDE', 'PDOCOTDES', 'PCERT1A', 'PCERT1ADE', 'PCERT1ADES',
       'PCERT1B', 'PCERT1BDE', 'PCERT1BDES', 'PCERT2', 'PCERT2DE', 'PCERT2DES',
       'PCERT4', 'PCERT4DE', 'PCERT4DES', 'PPBACC', 'PPBACCDE', 'PPBACCDES',
       'PPMAST', 'PPMASTDE', 'PPMASTDES'],
      dtype='object')

In [ ]:

# Convert CIP float values to strings and remove  summary CIPS ending with  ".0" by institution
mask = de_degrees1822['CIPCODE'].astype(str).str.endswith('.0')
summarycips = de_degrees1822[mask]
de_degrees_1822=de_degrees1822[~mask]


In [ ]:
# summarycips['CIPCODE']
# completes_1822.head()
de_degrees_1822['CIPCODE']

1           1.0999
2           1.1001
3           1.9999
5           3.0599
7           4.0301
            ...   
1205336    52.1101
1205337    52.1302
1205338    52.1401
1205339    52.1501
1205341    54.0101
Name: CIPCODE, Length: 886086, dtype: float64

Bureau of Labor & Statistics Occupational U.S. CES (2018-2022) and 2032 projections based on Employment categories (SOC2018 codes)

In [ ]:
#read in bls oes 2018-2022 files and create df of combined occ_code, tot_emp variables for 2018-2022

def combine_bls_files(file_paths, output_file):

    dfs = []

    # # Iterate over files in the directory
    for file in file_paths:

      # Read  file and select columns
      df = pd.read_excel(file, sheet_name=0, usecols=["OCC_CODE", "TOT_EMP"])

      parts = file.split('_')
      for part in parts:
        if part.startswith('M') and part[1:].isdigit():  # Check if part starts with 'M' and the rest are digits
          year = int(part[1:])  # Extract digits after 'M'

      df['YEAR'] = year

      dfs.append(df)

     # Concatenate all dfs in dfs list
      combined_df = pd.concat(dfs, ignore_index=True)

    # Save combined dfs to new excel file
    combined_df.to_excel(output_file, index=False)
    print(f"Combined data saved to {output_file}")
    return combined_df


# directory = '/cont/content/drive/MyDrive/Capstone_Winter2024/Data Sources/BLS Occupation Data/national_M2018_dl.xlsxent/drive/MyDrive/Capstone_Winter2024/Data Sources/Bureau of Labor Statistics/BLS Occupation Data'
file_paths = ['/content/drive/MyDrive/Capstone_Winter2024/TABULAR DATA/Bureau Labor Statistics Data/BLS Occupation Data/national_M2018_dl.xlsx',
              '/content/drive/MyDrive/Capstone_Winter2024/TABULAR DATA/Bureau Labor Statistics Data/BLS Occupation Data/national_M2019_dl.xlsx',
              '/content/drive/MyDrive/Capstone_Winter2024/TABULAR DATA/Bureau Labor Statistics Data/BLS Occupation Data/national_M2020_dl.xlsx',
              '/content/drive/MyDrive/Capstone_Winter2024/TABULAR DATA/Bureau Labor Statistics Data/BLS Occupation Data/national_M2021_dl.xlsx',
              '/content/drive/MyDrive/Capstone_Winter2024/TABULAR DATA/Bureau Labor Statistics Data/BLS Occupation Data/national_M2022_dl.xlsx']
output_file = "bls_1822.xlsx"
bls_1822 = combine_bls_files(file_paths, output_file)
bls_1822 =bls_1822.rename( columns ={'OCC_CODE': 'SOC2018Code'})

# execution_time = timeit.timeit(lambda: combine_bls_files(directory, output_file), number=1)
# print(f"Execution time: {execution_time} seconds")


Combined data saved to bls_1822.xlsx


In [ ]:
bls_1822.head()

,SOC2018Code,TOT_EMP,YEAR
0,00-0000,144733270,2018
1,11-0000,7616650,2018
2,11-1000,2535640,2018
3,11-1010,195530,2018
4,11-1011,195530,2018


In [ ]:
# #read in BLS 2022 employment projection file Table 1.7 and Table 1.10
# projections = pd.read_excel('/content/drive/MyDrive/Capstone_Winter2024/Data Sources/BLS Modeled Data/occupation 2022-2032 .xlsx', header = 1)
# projections

from openpyxl import load_workbook
workbook = load_workbook("/content/drive/MyDrive/Capstone_Winter2024/TABULAR DATA/Bureau Labor Statistics Data/BLS Modeled Data/occupation 2022-2032.xlsx")
sheet1 = workbook.worksheets[8]
sheet2 = workbook.worksheets[11]


# Read data from each sheet into DataFrames
df1 = pd.DataFrame(sheet1.values)
df1 = df1.iloc[1:]
df1.columns = df1.iloc[0]  # Set the first row as column names
df1 = df1[1:]


df2 = pd.DataFrame(sheet2.values)
df2 = df2.iloc[1:]
df2.columns = df2.iloc[0]  # Set the first row as column names
df2 = df2[1:]

# # Define columns you need from each sheet
columns_sheet1 = ["2022 National Employment Matrix code", "Employment, 2032", 'Employment change, percent, 2022-32', 'Occupational openings, 2022-32 annual average',
              ]  # columns needed
columns_sheet2 = ['2022 National Employment Matrix code','Labor force exits, 2022-32 annual average',"Occupational transfers, 2022-32 annual average",
                  'Total occupational separations, 2022-32 annual average', 'Occupational openings, 2022-32 annual average']  # columns needed

# Extract desired columns from each DataFrame
df1_selected = df1[columns_sheet1]
df2_selected = df2[columns_sheet2]

# # rename columns or perform additional data processing here
df1_selected = df1_selected.rename(columns ={"2022 National Employment Matrix code": 'SOC2018Code',
                              "Employment, 2032": 'EMP 2032',
                              'Employment change, percent, 2022-32':'EMP % CHANGE 22-32',
                              'Occupational openings, 2022-32 annual average': 'OCC_OPEN ANN AVG 22-32',
                              } )

df2_selected =df2_selected.rename(columns ={"2022 National Employment Matrix code": 'SOC2018Code',
                              'Labor force exits, 2022-32 annual average': 'LF EXIT ANN AVG 22-32',
                              "Occupational transfers, 2022-32 annual average" :"OCC XFER ANN AVG 22-32",
                              'Total occupational separations, 2022-32 annual average': "TOT OCC SEP ANN AVG 22-32",
                              'Occupational openings, 2022-32 annual average' :'OCC OPEN ANN AVG 22-32'})

bls_projections = pd.merge(df1_selected, df2_selected, on='SOC2018Code', how = 'inner', left_index=False, right_index=False)
bls_projections = bls_projections.dropna()
#original columns are in thousands, convert
conver_cols = [ 'EMP 2032',
       'OCC_OPEN ANN AVG 22-32',  'LF EXIT ANN AVG 22-32',
       'OCC XFER ANN AVG 22-32', 'TOT OCC SEP ANN AVG 22-32',
       'OCC OPEN ANN AVG 22-32']
for col in conver_cols:
  bls_projections[col]*=1000




In [ ]:
bls_projections

1,SOC2018Code,EMP 2032,EMP % CHANGE 22-32,OCC_OPEN ANN AVG 22-32,LF EXIT ANN AVG 22-32,OCC XFER ANN AVG 22-32,TOT OCC SEP ANN AVG 22-32,OCC OPEN ANN AVG 22-32
0,00-0000,169148100.0,2.8,18566500.0,8018300.0,10081700.0,18100000,18566500.0
1,11-0000,13257700.0,5.4,1053100.0,389200.0,595600.0,984800.0,1053100.0
2,11-1000,3958000,3.3,315000,98800.0,203600.0,302400.0,315000
3,11-1011,257000,-8.2,15300.0,8300.0,9300.0,17600.0,15300.0
4,11-1021,3655100.0,4.2,296300.0,89200.0,192400.0,281500.0,296300.0
...,...,...,...,...,...,...,...,...
1108,53-7072,11600.0,3.6,1200.0,400.0,700.0,1100.0,1200.0
1109,53-7073,15900.0,-2.2,1600.0,600.0,1100.0,1600.0,1600.0
1110,53-7081,151100.0,3.2,18600.0,6800.0,11400.0,18200.0,18600.0
1111,53-7121,13500.0,1.6,1600.0,700.0,900.0,1600.0,1600.0


In [ ]:
#merge with bls_1822 and bls_projections on SOC2018Code
merged_bls_1822=pd.merge(bls_1822, bls_projections, on ='SOC2018Code', how = 'inner')
merged_bls_1822 = merged_bls_1822.rename(columns = {'EMP % CHANGE 22-32': 'EMP % CHANGE 2032'})
# merged_bls_1822.dtypes
merged_bls_1822['EMP 2032']  = pd.to_numeric(merged_bls_1822['EMP 2032'] , errors='coerce')
merged_bls_1822['TOT_EMP'] = pd.to_numeric(merged_bls_1822['TOT_EMP'], errors='coerce')

merged_bls_1822['EMP % CHANGE 2032'] = round( ((merged_bls_1822['EMP 2032'] - merged_bls_1822['TOT_EMP'] )/merged_bls_1822['TOT_EMP']) *100, 0)


In [ ]:
merged_bls_1822.head()

,SOC2018Code,TOT_EMP,YEAR,EMP 2032,EMP % CHANGE 2032,OCC_OPEN ANN AVG 22-32,LF EXIT ANN AVG 22-32,OCC XFER ANN AVG 22-32,TOT OCC SEP ANN AVG 22-32,OCC OPEN ANN AVG 22-32
0,00-0000,144733270.0,2018,169148100.0,17.0,18566500.0,8018300.0,10081700.0,18100000,18566500.0
1,00-0000,146875480.0,2019,169148100.0,15.0,18566500.0,8018300.0,10081700.0,18100000,18566500.0
2,00-0000,139099570.0,2020,169148100.0,22.0,18566500.0,8018300.0,10081700.0,18100000,18566500.0
3,00-0000,140886310.0,2021,169148100.0,20.0,18566500.0,8018300.0,10081700.0,18100000,18566500.0
4,00-0000,147886000.0,2022,169148100.0,14.0,18566500.0,8018300.0,10081700.0,18100000,18566500.0


In [ ]:
#get median annual wage for each sector by year

def get_med_wage(file_paths):
    dfs = []

    for file in file_paths:
        # Extract year from file name
        year = int(file.split(' ')[-1].split('-')[0])  # Extract the starting year from the file name

        # Read the file and select columns
        df = pd.read_excel(file, sheet_name="Table 1.7", header=1)


        # Filter columns containing median annual wage
        med_wage_columns = [col for col in df.columns if 'Median annual wage' in col]
        # print (med_wage_columns)
        soc_columns = [col for col in df.columns if 'National Employment Matrix code' in col]


    #     # Select relevant columns and rename them
        df = df[soc_columns + med_wage_columns]
        df.rename(columns={med_wage_columns[0]: 'Median_annual_wage', soc_columns[0]: 'SOC2018Code'}, inplace=True)
              # Add 'YEAR' column
        df['YEAR'] = year
        dfs.append(df)

    # Concatenate all dfs in dfs list
    combined_dfs = pd.concat(dfs, ignore_index=True)
    combined_dfs = combined_dfs.dropna()
    return combined_dfs

# Example usage:
file_paths = ['/content/drive/MyDrive/Capstone_Winter2024/TABULAR DATA/Bureau Labor Statistics Data/BLS Modeled Data/occupation 2018-2028.xlsx',
              '/content/drive/MyDrive/Capstone_Winter2024/TABULAR DATA/Bureau Labor Statistics Data/BLS Modeled Data/occupation 2019-2029.xlsx',
              '/content/drive/MyDrive/Capstone_Winter2024/TABULAR DATA/Bureau Labor Statistics Data/BLS Modeled Data/occupation 2020-2030.xlsx',
              '/content/drive/MyDrive/Capstone_Winter2024/TABULAR DATA/Bureau Labor Statistics Data/BLS Modeled Data/occupation 2021-2031.xlsx',
              '/content/drive/MyDrive/Capstone_Winter2024/TABULAR DATA/Bureau Labor Statistics Data/BLS Modeled Data/occupation 2022-2032.xlsx']

med_wage = get_med_wage(file_paths)




In [ ]:
med_wage

,SOC2018Code,Median_annual_wage,YEAR
1,00-0000,39810,2018
2,11-0000,105660,2018
3,11-1000,103230,2018
4,11-1011,184460,2018
5,11-1021,100780,2018
...,...,...,...
5411,53-7072,55860,2022
5412,53-7073,63280,2022
5413,53-7081,43540,2022
5414,53-7121,53930,2022


In [ ]:
# #check median annual wage values
# med_wage['Median_annual_wage'].unique().tolist()

In [ ]:
#clean median  wage column - string characters appear

import re
def clean_and_convert(value):
  # Convert NaN values to empty strings
    if pd.isna(value):
        return value
    # Replace "—" with NaN
    if str(value) == "—":
        return pd.NA
    # Remove non-numeric characters and decimal points
    cleaned_value = re.sub(r'[^\d]', '', str(value).split('(')[0])
    # Convert to integer
    return int(cleaned_value)

# Apply function to each value in the column
med_wage['Median_annual_wage'] = med_wage['Median_annual_wage'].apply(clean_and_convert)


In [ ]:
#Merge median annual wage based on year and SOC Code into merged_bls_1822
merged2_bls_1822 = pd.merge(merged_bls_1822, med_wage, on=['SOC2018Code', 'YEAR'], how='inner')

In [ ]:
#merge bls labor data with codes crosswalk data on common SOC2018Code to merge in CIPCODES for matching to IPEDs data
merged3_bls_1822= pd.merge(merged2_bls_1822, codes, on ='SOC2018Code', how = 'inner')

In [ ]:
#move CIPCODE column to beginning
column_to_move = merged3_bls_1822.pop("CIPCODE")
merged3_bls_1822.insert(0, "CIPCODE", column_to_move)

In [ ]:
merged3_bls_1822

,CIPCODE,SOC2018Code,TOT_EMP,YEAR,EMP 2032,EMP % CHANGE 2032,OCC_OPEN ANN AVG 22-32,LF EXIT ANN AVG 22-32,OCC XFER ANN AVG 22-32,TOT OCC SEP ANN AVG 22-32,OCC OPEN ANN AVG 22-32,Median_annual_wage
0,44.0401,11-1011,195530.0,2018,257000.0,31.0,15300.0,8300.0,9300.0,17600.0,15300.0,184460
1,52.0101,11-1011,195530.0,2018,257000.0,31.0,15300.0,8300.0,9300.0,17600.0,15300.0,184460
2,52.0201,11-1011,195530.0,2018,257000.0,31.0,15300.0,8300.0,9300.0,17600.0,15300.0,184460
3,52.0206,11-1011,195530.0,2018,257000.0,31.0,15300.0,8300.0,9300.0,17600.0,15300.0,184460
4,52.0701,11-1011,195530.0,2018,257000.0,31.0,15300.0,8300.0,9300.0,17600.0,15300.0,184460
...,...,...,...,...,...,...,...,...,...,...,...,...
26549,49.0205,53-3053,201070.0,2022,235900.0,17.0,32600.0,19400.0,11400.0,30900.0,32600.0,32800
26550,99.9999,53-3054,13950.0,2021,213700.0,1432.0,22800.0,9900.0,9200.0,19200.0,22800.0,29310
26551,99.9999,53-3054,13820.0,2022,213700.0,1446.0,22800.0,9900.0,9200.0,19200.0,22800.0,30670
26552,99.9999,53-6032,12170.0,2021,20800.0,71.0,3000,1100.0,1900.0,3000,3000,36390


In [ ]:
# #check values in median annual wage column
# merged3_bls_1822['Median_annual_wage'].unique().tolist()

In [ ]:
#check for nan
merged3_bls_1822.isnull().sum()

CIPCODE                        0
SOC2018Code                    0
TOT_EMP                        1
YEAR                           0
EMP 2032                       0
EMP % CHANGE 2032              1
OCC_OPEN ANN AVG 22-32         0
LF EXIT ANN AVG 22-32          0
OCC XFER ANN AVG 22-32         0
TOT OCC SEP ANN AVG 22-32      0
OCC OPEN ANN AVG 22-32         0
Median_annual_wage           217
dtype: int64

In [ ]:
# Finding rows with NaN values
rows_with_nan = merged3_bls_1822[merged3_bls_1822.isnull().any(axis=1)]
rows_with_nan.tail(50)

,CIPCODE,SOC2018Code,TOT_EMP,YEAR,EMP 2032,EMP % CHANGE 2032,OCC_OPEN ANN AVG 22-32,LF EXIT ANN AVG 22-32,OCC XFER ANN AVG 22-32,TOT OCC SEP ANN AVG 22-32,OCC OPEN ANN AVG 22-32,Median_annual_wage
20521,51.0908,29-2099,125270.0,2018,179100.0,43.0,12600.0,6200.0,5400.0,11500.0,12600.0,<NA>
20522,51.0914,29-2099,125270.0,2018,179100.0,43.0,12600.0,6200.0,5400.0,11500.0,12600.0,<NA>
20523,51.0915,29-2099,125270.0,2018,179100.0,43.0,12600.0,6200.0,5400.0,11500.0,12600.0,<NA>
20524,51.0916,29-2099,125270.0,2018,179100.0,43.0,12600.0,6200.0,5400.0,11500.0,12600.0,<NA>
20525,51.0917,29-2099,125270.0,2018,179100.0,43.0,12600.0,6200.0,5400.0,11500.0,12600.0,<NA>
20526,51.0921,29-2099,125270.0,2018,179100.0,43.0,12600.0,6200.0,5400.0,11500.0,12600.0,<NA>
20527,51.0922,29-2099,125270.0,2018,179100.0,43.0,12600.0,6200.0,5400.0,11500.0,12600.0,<NA>
20528,51.0923,29-2099,125270.0,2018,179100.0,43.0,12600.0,6200.0,5400.0,11500.0,12600.0,<NA>
20529,51.0999,29-2099,125270.0,2018,179100.0,43.0,12600.0,6200.0,5400.0,11500.0,12600.0,<NA>
20530,51.1011,29-2099,125270.0,2018,179100.0,43.0,12600.0,6200.0,5400.0,11500.0,12600.0,<NA>


In [ ]:

# rows_with_nan['SOC2018Code'].unique().tolist()
# rows_with_nan[rows_with_nan['SOC2018Code']== '27-2091']

In [ ]:
#replace NaN values for years using closest year (after) median-annual wage.

# '''
#  '13-2051' 2018  $93,330
#  '13-2099' 2018 $73,535
#  '19-4099' 2020  $52,460
#  '27-2011',2018-2022  Actors drop
#  '27-2031',2018-2022  Dances drop
#  '27-2042',2018-2022  Musicians drop
#  '27-2099',2018,21,22 Entertainers drop
#  '27-3099', 2018  $49730
#  '29-2099', 2018 $51870
#  '41-9012', 2020,2021  Models drop
#  '43-2099', 2018  46680
#  '47-5049', 2019 61260
#  '47-5099', 2018 48140
#  '49-9099', 2018 42480
#  '51-9199', 2020  31420
#  '53-7199', 2020  32850
#  '15-1299', 2020  92870
#  '47-5022', 2020  45150
#  '51-9161', 2020 42260
#  '51-9162', 2020 57740
#  '27-2091', 2021, 2022  Disc jockey drop'''

"\n '13-2051' 2018  first appears 2021 $93,330\n '13-2099' 2018 $73,535\n '19-4099' 2020  $52,460\n '27-2011',2018-2022  Actors drop\n '27-2031',2018-2022  Dances drop\n '27-2042',2018-2022  Musicians drop\n '27-2099',2018,21,22 Entertainers drop\n '27-3099', 2018  $49730\n '29-2099', 2018 $51870\n '41-9012', 2020,2021  Models drop\n '43-2099', 2018  46680\n '47-5049', 2019 61260\n '47-5099', 2018 48140\n '49-9099', 2018 42480\n '51-9199', 2020  31420\n '53-7199', 2020  32850\n '15-1299', 2020  92870\n '47-5022', 2020  45150\n '51-9161', 2020 42260\n '51-9162', 2020 57740\n '27-2091', 2021, 2022  Disc jockey drop"

In [ ]:

#drop those where no formale educational credential necessarily required
#  '27-2091', 2021, 2022  Disc jockey drop'
#  '41-9012', 2020,2021  Models drop
#  '27-2011',2018-2022  Actors drop
#  '27-2031',2018-2022  Dances drop
#  '27-2042',2018-2022  Musicians drop
#  '27-2099',2018,21,22 Entertainers drop
socs_exclude= ['27-2011','27-2031','27-2042','27-2099', '41-9012','27-2091']
merged4_bls_1822 = merged3_bls_1822[~merged3_bls_1822['SOC2018Code'].isin(socs_exclude)]



In [ ]:

#replace NaN values for years using next closest year (after) median-annual wage.
merged4_bls_1822= merged4_bls_1822.copy()
replacement_dict = {
    '13-2051': '93330',
    '13-2099': '73,535',
    '19-4099': '52,460',
    '27-3099': '49730',
    '29-2099': '51870',
    '43-2099': '46680',
    '47-5049': '61260',
    '47-5099': '48140',
    '49-9099': '42480',
    '51-9199': '31420',
    '53-7199': '32850',
    '15-1299': '92870',
    '47-5022': '45150',
    '51-9161': '42260',
    '51-9162': '57740'
}


# Fill NaN values in 'Median_annual_wage' column using replacement_dict
merged4_bls_1822['Median_annual_wage'] = merged4_bls_1822.apply(lambda row: replacement_dict.get(row['SOC2018Code'], row['Median_annual_wage']), axis=1)

In [ ]:
merged4_bls_1822.head()
merged4_bls_1822.isnull().sum()
# rows_with_nan2 = merged4_bls_1822[merged4_bls_1822.isnull().any(axis=1)]
# rows_with_nan2.head(50)

CIPCODE                      0
SOC2018Code                  0
TOT_EMP                      1
YEAR                         0
EMP 2032                     0
EMP % CHANGE 2032            1
OCC_OPEN ANN AVG 22-32       0
LF EXIT ANN AVG 22-32        0
OCC XFER ANN AVG 22-32       0
TOT OCC SEP ANN AVG 22-32    0
OCC OPEN ANN AVG 22-32       0
Median_annual_wage           0
dtype: int64

In [ ]:
# # Finding rows with NaN values
rows_with_nan = merged4_bls_1822[merged4_bls_1822.isnull().any(axis=1)]
rows_with_nan

,CIPCODE,SOC2018Code,TOT_EMP,YEAR,EMP 2032,EMP % CHANGE 2032,OCC_OPEN ANN AVG 22-32,LF EXIT ANN AVG 22-32,OCC XFER ANN AVG 22-32,TOT OCC SEP ANN AVG 22-32,OCC OPEN ANN AVG 22-32,Median_annual_wage
2973,99.9999,13-1074,NaN,2020,1700.0,NaN,200.0,100.0,100.0,100.0,200.0,47770


In [ ]:
merged4_bls_1822 = merged4_bls_1822.dropna()

In [ ]:
merged4_bls_1822.isna().sum()

CIPCODE                      0
SOC2018Code                  0
TOT_EMP                      0
YEAR                         0
EMP 2032                     0
EMP % CHANGE 2032            0
OCC_OPEN ANN AVG 22-32       0
LF EXIT ANN AVG 22-32        0
OCC XFER ANN AVG 22-32       0
TOT OCC SEP ANN AVG 22-32    0
OCC OPEN ANN AVG 22-32       0
Median_annual_wage           0
dtype: int64

#Get distance education exclusive (DE EXC) enrollment data for each year at institution summary level (EFDELEV==1). EFDENON != EFDETOT


In [ ]:
#read in distance enrollment dataset = dist_enrollment_1822 (all years)
enrolled = pd.read_csv('/content/drive/MyDrive/Capstone_Winter2024/PROJECT DATASETS/dist_enrollment18_22.csv')

In [ ]:
enrolled.columns

Index(['UNITID', 'INSTNM', 'YEAR', 'IALIAS', 'STABBR', 'OBEREG', 'ICLEVEL',
       'CONTROL', 'DEGGRANT', 'LONGITUD', 'LATITUDE', 'EFDELEV', 'EFDETOT',
       'EFDEEXC', 'EFDESOM', 'EFDENON', 'EFDEEX1', 'EFDEEX2', 'EFDEEX3',
       'EFDEEX4', 'EFDEEX5 '],
      dtype='object')

In [ ]:
#filter for  EFDELEV = 1 (Summary line for each institution)

enrolled_sum = enrolled[ enrolled['EFDELEV'] == 1]


In [ ]:
enrolled_sum2 = enrolled_sum[enrolled_sum['EFDENON'] != enrolled_sum['EFDETOT']]
enrolled_sum2


,UNITID,INSTNM,YEAR,IALIAS,STABBR,OBEREG,ICLEVEL,CONTROL,DEGGRANT,LONGITUD,...,EFDELEV,EFDETOT,EFDEEXC,EFDESOM,EFDENON,EFDEEX1,EFDEEX2,EFDEEX3,EFDEEX4,EFDEEX5
0,100654,Alabama A & M University,2018,AAMU,AL,5,1,1,1,-86.568502,...,1,6106,120,1089,4897,87.0,16.0,17.0,0.0,0.0
5,100663,University of Alabama at Birmingham,2018,,AL,5,1,1,1,-86.799345,...,1,21923,5603,6935,9385,3915.0,1674.0,0.0,7.0,7.0
10,100690,Amridge University,2018,Southern Christian University |Regions University,AL,5,1,2,1,-86.174010,...,1,755,755,0,0,441.0,253.0,0.0,0.0,61.0
14,100706,University of Alabama in Huntsville,2018,UAH |University of Alabama Huntsville,AL,5,1,1,1,-86.640449,...,1,9736,638,2044,7054,428.0,210.0,0.0,0.0,0.0
19,100724,Alabama State University,2018,,AL,5,1,1,1,-86.295677,...,1,4413,437,296,3680,316.0,110.0,11.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111778,498456,Arizona College-Ontario,2022,,CA,8,1,3,1,-117.578071,...,1,210,0,210,0,NaN,NaN,NaN,NaN,NaN
111784,498483,Galen Health Institutes-Pikeville,2022,,KY,5,2,3,1,-82.530209,...,1,26,0,26,0,NaN,NaN,NaN,NaN,NaN
111787,498553,Stellar Career College,2022,,IL,3,3,3,2,-87.634151,...,1,141,0,141,0,NaN,NaN,NaN,NaN,NaN
111790,498562,Commonwealth University of Pennsylvania,2022,,PA,2,1,1,1,-76.447844,...,1,12074,1579,6100,4395,1428.0,147.0,1.0,3.0,0.0


In [ ]:
enrolled_sum2.columns
# '''['UNITID', 'INSTNM', 'YEAR', 'IALIAS', 'STABBR', 'OBEREG', 'ICLEVEL',
#        'CONTROL', 'HLOFFER', 'DEGGRANT', 'PSEFLAG', 'LONGITUD', 'LATITUDE',
#        'EFDELEV', 'EFDETOT', 'EFDEEXC', 'EFDESOM', 'EFDENON', 'EFDEEX1',
#        'EFDEEX2', 'EFDEEX3', 'EFDEEX4', 'EFDEEX5 '],'''

Index(['UNITID', 'INSTNM', 'YEAR', 'IALIAS', 'STABBR', 'OBEREG', 'ICLEVEL',
       'CONTROL', 'DEGGRANT', 'LONGITUD', 'LATITUDE', 'EFDELEV', 'EFDETOT',
       'EFDEEXC', 'EFDESOM', 'EFDENON', 'EFDEEX1', 'EFDEEX2', 'EFDEEX3',
       'EFDEEX4', 'EFDEEX5 '],
      dtype='object')

In [ ]:
#keep columns of interest for time forecasting
#UNITID/YER/STABBR for merging, Target = EFDEEXC
cols_to_keep=['UNITID', 'YEAR','STABBR', 'EFDEEXC','EFDESOM']
enrolled_de = enrolled_sum2[cols_to_keep]

len(enrolled_de) #17413

17413

In [ ]:
enrolled_de


,UNITID,YEAR,STABBR,EFDEEXC,EFDESOM
0,100654,2018,AL,120,1089
5,100663,2018,AL,5603,6935
10,100690,2018,AL,755,0
14,100706,2018,AL,638,2044
19,100724,2018,AL,437,296
...,...,...,...,...,...
111778,498456,2022,CA,0,210
111784,498483,2022,KY,0,26
111787,498553,2022,IL,0,141
111790,498562,2022,PA,1579,6100




#Merge enrolled_de (enrollment exclusvely and some DE) with de_degrees_1822 (DE degrees offered)

In [ ]:
de_degrees_1822.columns

Index(['UNITID', 'INSTNM', 'YEAR', 'IALIAS', 'STABBR', 'OBEREG', 'ICLEVEL',
       'CONTROL', 'DEGGRANT', 'LONGITUD', 'LATITUDE', 'CIPCODE', 'PTOTAL',
       'PTOTALDE', 'PTOTALDES', 'PASSOC', 'PASSOCDE', 'PASSOCDES', 'PBACHL',
       'PBACHLDE', 'PBACHLDES', 'PMASTR', 'PMASTRDE', 'PMASTRDES', 'PDOCRS',
       'PDOCRSDE', 'PDOCRSDES', 'PDOCPP', 'PDOCPPDE', 'PDOCPPDES', 'PDOCOT',
       'PDOCOTDE', 'PDOCOTDES', 'PCERT1A', 'PCERT1ADE', 'PCERT1ADES',
       'PCERT1B', 'PCERT1BDE', 'PCERT1BDES', 'PCERT2', 'PCERT2DE', 'PCERT2DES',
       'PCERT4', 'PCERT4DE', 'PCERT4DES', 'PPBACC', 'PPBACCDE', 'PPBACCDES',
       'PPMAST', 'PPMASTDE', 'PPMASTDES'],
      dtype='object')

In [ ]:
degrees_keep= ['UNITID', 'INSTNM', 'YEAR', 'STABBR',
       'CIPCODE', 'PTOTAL', 'PTOTALDE', ]
#PTOTALDES not used as variable of interest as it was not collected by IPEDS in 2018 & 2019.


In [ ]:
de_degrees2_1822 =de_degrees_1822[degrees_keep]
# len(de_degrees2_1822) #886086
de_degrees2_1822.head()

,UNITID,INSTNM,YEAR,STABBR,CIPCODE,PTOTAL,PTOTALDE
1,100654,Alabama A & M University,2019,AL,1.0999,1,0
2,100654,Alabama A & M University,2019,AL,1.1001,3,0
3,100654,Alabama A & M University,2019,AL,1.9999,3,0
5,100654,Alabama A & M University,2019,AL,3.0599,1,0
7,100654,Alabama A & M University,2019,AL,4.0301,2,0


In [ ]:
#filter for PTOTLAL DE > 0 & PTOTALDES is not null (will be the case of years 2018 * 2019 where this was not tracked by IPEDS)
de_degrees3_1822 = de_degrees2_1822[de_degrees2_1822['PTOTALDE'] > 0]
# de_degrees3_1822 = de_degrees2_1822[(de_degrees2_1822['PTOTALDE'] > 0) & (de_degrees2_1822['PTOTALDES'].notnull()) ]
de_degrees3_1822.head()

,UNITID,INSTNM,YEAR,STABBR,CIPCODE,PTOTAL,PTOTALDE
13,100654,Alabama A & M University,2019,AL,11.0101,2,1
16,100654,Alabama A & M University,2019,AL,13.0401,1,1
21,100654,Alabama A & M University,2019,AL,13.1210,2,1
64,100654,Alabama A & M University,2019,AL,52.0201,2,1
80,100663,University of Alabama at Birmingham,2019,AL,13.0101,1,1


In [ ]:
# de_degrees3_1822[de_degrees3_1822['UNITID']==100690]
de_degrees3_1822.head()
# len(de_degrees3_1822)  #106131

,UNITID,INSTNM,YEAR,STABBR,CIPCODE,PTOTAL,PTOTALDE
13,100654,Alabama A & M University,2019,AL,11.0101,2,1
16,100654,Alabama A & M University,2019,AL,13.0401,1,1
21,100654,Alabama A & M University,2019,AL,13.1210,2,1
64,100654,Alabama A & M University,2019,AL,52.0201,2,1
80,100663,University of Alabama at Birmingham,2019,AL,13.0101,1,1


In [ ]:
#merge de_degrees3_1822 with enrolled_de_exc  on UNITID, YEAR, STABBR
df = pd.merge(de_degrees3_1822, enrolled_de_exc, on=['UNITID', 'YEAR', 'STABBR'], how='inner')



In [ ]:
df.head()
# len(df) #105547
# df.isnull().sum() #0

,UNITID,INSTNM,YEAR,STABBR,CIPCODE,PTOTAL,PTOTALDE,EFDEEXC,EFDESOM
0,100654,Alabama A & M University,2019,AL,11.0101,2,1,102,1213
1,100654,Alabama A & M University,2019,AL,13.0401,1,1,102,1213
2,100654,Alabama A & M University,2019,AL,13.1210,2,1,102,1213
3,100654,Alabama A & M University,2019,AL,52.0201,2,1,102,1213
4,100654,Alabama A & M University,2019,AL,11.0101,2,1,102,1213


#MERGE IN BLS DATASET (merged4_bls_1822)

In [ ]:
merged4_bls_1822.columns

Index(['CIPCODE', 'SOC2018Code', 'TOT_EMP', 'YEAR', 'EMP 2032',
       'EMP % CHANGE 2032', 'OCC_OPEN ANN AVG 22-32', 'LF EXIT ANN AVG 22-32',
       'OCC XFER ANN AVG 22-32', 'TOT OCC SEP ANN AVG 22-32',
       'OCC OPEN ANN AVG 22-32', 'Median_annual_wage'],
      dtype='object')

In [ ]:
#merge bls and df datasets based on common CIPCode

df2=pd.merge(df, merged4_bls_1822, on =['CIPCODE','YEAR'])

In [ ]:
df2.isnull().sum()

UNITID                       0
INSTNM                       0
YEAR                         0
STABBR                       0
CIPCODE                      0
PTOTAL                       0
PTOTALDE                     0
EFDEEXC                      0
EFDESOM                      0
SOC2018Code                  0
TOT_EMP                      0
EMP 2032                     0
EMP % CHANGE 2032            0
OCC_OPEN ANN AVG 22-32       0
LF EXIT ANN AVG 22-32        0
OCC XFER ANN AVG 22-32       0
TOT OCC SEP ANN AVG 22-32    0
OCC OPEN ANN AVG 22-32       0
Median_annual_wage           0
dtype: int64

,UNITID,INSTNM,YEAR,STABBR,CIPCODE,PTOTAL,PTOTALDE,EFDEEXC,EFDESOM,SOC2018Code,TOT_EMP,EMP 2032,EMP % CHANGE 2032,OCC_OPEN ANN AVG 22-32,LF EXIT ANN AVG 22-32,OCC XFER ANN AVG 22-32,TOT OCC SEP ANN AVG 22-32,OCC OPEN ANN AVG 22-32,Median_annual_wage


#Add state unemployment rates for each year to df

In [ ]:
#merge in unemployment rate on STATE ABBR
ur=pd.read_excel('/content/drive/MyDrive/Capstone_Winter2024/TABULAR DATA/Bureau Labor Statistics Data/StateUR18_22.xlsx')

In [ ]:
ur

,STABBR,YEAR,UR
0,AL,2018,3.9
1,AK,2018,6.0
2,AZ,2018,4.8
3,AR,2018,3.7
4,CA,2018,4.2
...,...,...,...
250,VA,2022,2.8
251,WA,2022,4.1
252,WV,2022,3.9
253,WI,2022,2.9


In [ ]:
# Merge df2 with ur based on STABBR
model_data= pd.merge(df2, ur, on=['STABBR','YEAR'])


In [ ]:
# model_data.isnull().sum()
# model_data.shape  #(432208, 20)

model_data

,UNITID,INSTNM,YEAR,STABBR,CIPCODE,PTOTAL,PTOTALDE,EFDEEXC,EFDESOM,SOC2018Code,TOT_EMP,EMP 2032,EMP % CHANGE 2032,OCC_OPEN ANN AVG 22-32,LF EXIT ANN AVG 22-32,OCC XFER ANN AVG 22-32,TOT OCC SEP ANN AVG 22-32,OCC OPEN ANN AVG 22-32,Median_annual_wage,UR
0,100654,Alabama A & M University,2019,AL,11.0101,2,1,102,1213,11-3021,433960.0,643300.0,48.0,46900.0,11700.0,26600.0,38300.0,46900.0,151150,3.2
1,100654,Alabama A & M University,2019,AL,11.0101,2,1,102,1213,25-1021,31800.0,44300.0,39.0,3600.0,1900.0,1500.0,3400.0,3600.0,85540,3.2
2,100654,Alabama A & M University,2019,AL,11.0101,2,1,102,1213,15-1211,589060.0,582600.0,-1.0,37600.0,13900.0,18500.0,32500.0,37600.0,93730,3.2
3,100654,Alabama A & M University,2019,AL,11.0101,2,1,102,1213,15-1221,30780.0,44800.0,46.0,3400.0,1000,1500.0,2500.0,3400.0,126830,3.2
4,100654,Alabama A & M University,2019,AL,11.0101,2,1,102,1213,15-1241,152420.0,186600.0,22.0,10200.0,3300.0,6300.0,9600.0,10200.0,116780,3.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432203,460349,Johnson & Wales University-Online,2022,RI,19.0505,2,2,2504,1,25-1192,2420.0,3000.0,24.0,200.0,100.0,100.0,200.0,200.0,76440,3.2
432204,460349,Johnson & Wales University-Online,2022,RI,19.0505,2,2,2504,1,29-1031,69880.0,83800.0,20.0,5600.0,2900.0,2200.0,5100.0,5600.0,66450,3.2
432205,460349,Johnson & Wales University-Online,2022,RI,19.0505,2,2,2504,1,35-1012,1169620.0,1281800.0,10.0,188900.0,63000,119900.0,182900.0,188900.0,37050,3.2
432206,460349,Johnson & Wales University-Online,2022,RI,19.0505,2,2,2504,1,35-2012,417780.0,443400.0,6.0,66600.0,29900.0,35800.0,65700.0,66600.0,32510,3.2


In [ ]:
model_data.to_csv('model_df.csv', index=False)
